In [1]:
import datetime

import numpy as np
import pandas as pd

from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.optimizers import SGD, Adam

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 5005)


In [2]:
with open('data/posture_train.txt', 'r') as train_file:
    train_df = pd.read_csv(train_file, header=None, delim_whitespace=True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,299.875,134.053,286.550,138.837,1,0,0,0,0,0,0,0,0
1,298.537,135.013,280.650,138.206,1,0,0,0,0,0,0,0,0
2,295.098,135.445,301.397,139.940,1,0,0,0,0,0,0,0,0
3,294.603,132.883,302.374,138.732,1,0,0,0,0,0,0,0,0
4,292.741,132.936,301.714,139.080,1,0,0,0,0,0,0,0,0


In [3]:
with open('data/posture_test.txt', 'r') as test_file:
    test_df = pd.read_csv(test_file, header=None, delim_whitespace=True)
test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,250.861,143.703,243.304,128.988,1,0,0,0,0,0,0,0,0
1,255.713,144.294,255.273,128.942,1,0,0,0,0,0,0,0,0
2,260.364,144.587,269.875,129.379,1,0,0,0,0,0,0,0,0
3,265.968,146.152,274.700,129.021,1,0,0,0,0,0,0,0,0
4,257.903,153.688,281.335,128.726,1,0,0,0,0,0,0,0,0


In [7]:
#train data
x = []
y = []
for index, row in train_df.iterrows():
    x.append(np.array(row[:4]))
    y.append(np.array(row[4:]))
x_train = np.array(x)
y_train = np.array(y)

array([ 299.875,  134.053,  286.55 ,  138.837])

In [9]:
#test data
x = []
y = []
for index, row in test_df.iterrows():
    x.append(np.array(row[:4]))
    y.append(np.array(row[4:]))
x_test = np.array(x)
y_test = np.array(y)

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [10]:
#model
model = Sequential()
model.add(Dense(input_dim=4, output_dim=1000))
model.add(Activation('sigmoid'))
model.add(Dense(output_dim=1000))
model.add(Activation('sigmoid'))
model.add(Dense(output_dim=9))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, batch_size=200, nb_epoch=10)

Epoch 1/10
18000/18000 [==============================] - 1s - loss: 0.4042 - acc: 0.8766     
Epoch 2/10
18000/18000 [==============================] - 1s - loss: 0.1344 - acc: 0.9559     
Epoch 3/10
18000/18000 [==============================] - 1s - loss: 0.1117 - acc: 0.9614     
Epoch 4/10
18000/18000 [==============================] - 1s - loss: 0.0992 - acc: 0.9659     
Epoch 5/10
18000/18000 [==============================] - 1s - loss: 0.1084 - acc: 0.9617     
Epoch 6/10
18000/18000 [==============================] - 1s - loss: 0.0938 - acc: 0.9671     
Epoch 7/10
18000/18000 [==============================] - 1s - loss: 0.0816 - acc: 0.9706     
Epoch 8/10
18000/18000 [==============================] - 1s - loss: 0.0809 - acc: 0.9698     
Epoch 9/10
18000/18000 [==============================] - 1s - loss: 0.0757 - acc: 0.9716     
Epoch 10/10
18000/18000 [==============================] - 1s - loss: 0.0859 - acc: 0.9673     


In [12]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', round(score[1], 4)*100)

Test score: 0.0738115009361
Test accuracy: 96.49


In [13]:
model_result = model.to_json()
time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
accuracy = str(int(score[1]*10000))
filename = 'model_{0}_acc{1}.json'.format(time, accuracy)
with open('result/'+filename+'.json', 'w') as file:
    file.write(model_result)
model.save_weights('result/'+filename+'.h5')